In [1]:
# import Dependecies and Setup
import pandas as pd

In [2]:
csvpath = 'Resources/purchase_data.csv'

df = pd.read_csv(csvpath)
df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count
- Total Number of Players

In [4]:
total_users = df['SN'].nunique()
total_users_df = pd.DataFrame([total_users], columns = ["Total Users"])
total_users_df

,Total Users
0,576


## Purchasing Analysis (Total)
- Number of Unique Items
- Average Purchase Price
- Total Number of Purchases
- Total Revenue

In [5]:
unique_items = df['Item ID'].nunique()
total_revenue = df['Price'].sum()
total_purchases = df['Price'].count()
average_price = round((total_revenue/total_purchases),2)

prices_df = pd.DataFrame({'Unique Items':[unique_items],'Average Price':[average_price],'Total Revenue':[total_revenue],'Total Purchases':[total_purchases]})

prices_df


,Unique Items,Average Price,Total Revenue,Total Purchases
0,183,3.05,2379.77,780


## Gender Demographics
- Percentage and Count of Male Players
- Percentage and Count of Female Players
- Percentage and Count of Other / Non-Disclosed

In [6]:
unique_users_df = df.drop_duplicates(subset='SN', keep='first', inplace=False)

genders_df = unique_users_df['Gender'].value_counts().to_frame()

genders_df.index.name = 'Gender'
genders_df = genders_df.rename(columns={'Gender':'Total'})

length = genders_df['Total'].sum()

genders_df['Percentage'] = round(((genders_df['Total']*100)/length),2)

genders_df



,Total,Percentage
Gender,,
Male,484,84.03
Female,81,14.06
Other / Non-Disclosed,11,1.91


## Purchasing Analysis (Gender)
- **The below each broken by gender**
    - Purchase Count
    - Average Purchase Price
    - Total Purchase Value
    - Average Purchase Total per Person by Gender

In [7]:
# Make purchase dataframe to do analysis
purchase_data = pd.DataFrame(df[['Gender','Price']])
# call unique genders df for analysis
genders_df = genders_df.reindex(['Female','Male','Other / Non-Disclosed'])

#avg prices
data = purchase_data.groupby('Gender').mean()
data['Average Purchase Price'] = round((data['Price']),2)

purchase_data.groupby('Gender').sum()
# Total $$
data['Total'] = purchase_data.groupby('Gender').sum()
# Total purchases
data['Purchase Count'] = purchase_data['Gender'].value_counts().to_frame()
# Total $$ divided by unique count
data['Avg Total per Person'] = round((data['Total']/genders_df['Total']),2)

data = data.drop(columns=['Price'])
data




,Average Purchase Price,Total,Purchase Count,Avg Total per Person
Gender,,,,
Female,3.20,361.94,113,4.47
Male,3.02,1967.64,652,4.07
Other / Non-Disclosed,3.35,50.19,15,4.56


In [8]:
bins = [0,9,14,19,24,29,34,39,1000]
labels = ['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']
df['bins'] = pd.cut(unique_users_df['Age'], bins, labels=labels)

bin_df = df['bins'].value_counts().to_frame().reindex(labels)
bin_df = bin_df.rename(columns = {'bins':'Totals'})
total = bin_df['Totals'].sum()
bin_df['% of Players'] = round(((bin_df['Totals']*100)/total),2)

bin_df

,Totals,% of Players
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


## Age Demographics
- **The below each broken into bins of 4 years (i.e. <10, 10-14, 15-19, etc.)**
    - Purchase Count
    - Average Purchase Price
    - Total Purchase Value
    - Average Purchase Total per Person by Age Group

In [9]:
bins = [0,9,14,19,24,29,34,39,1000]
labels = ['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']
df['bins'] = pd.cut(df['Age'], bins, labels=labels)

bin_count_df = df['bins'].value_counts().to_frame().reindex(labels)
bin_count_df = bin_count_df.rename(columns = {'bins':'Purchase Count'})
total = bin_count_df['Purchase Count'].sum()

avg_price = df.groupby('bins')['Price'].mean()
total_price = df.groupby('bins')['Price'].sum()
avg_tot_per_person = total_price/bin_df['Totals']

bin_count_df['Average Purchase Price'] = round(avg_price,2)
bin_count_df['Total Purchase Value']=total_price
bin_count_df['Avg Total Purchase per Person']= round(avg_tot_per_person,2)
bin_count_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
<10,23,3.35,77.13,4.54
10-14,28,2.96,82.78,3.76
15-19,136,3.04,412.89,3.86
20-24,365,3.05,1114.06,4.32
25-29,101,2.90,293.00,3.81
30-34,73,2.93,214.00,4.12
35-39,41,3.60,147.67,4.76
40+,13,2.94,38.24,3.19


## Top Spenders
- **Identify the the top 5 spenders in the game by total purchase value, then list (in a table):**
    - SN
    - Purchase Count
    - Average Purchase Price
    - Total Purchase Value

In [121]:
most_occurences = df.groupby('SN').count().sort_values('Purchase ID', ascending=False).reset_index()
sn_groups = df.groupby('SN').sum().reset_index()
sn_groups = sn_groups.sort_values('Price', ascending=False)


most_occurences = most_occurences.rename(columns={'Purchase ID':'Count'})
most_occurences = most_occurences.drop(columns=['Age','Gender','Item ID','Item Name','Price','bins'])

sn_purchases_df = sn_groups.merge(most_occurences, how='outer', on='SN')
sn_purchases_df['Avg Purchase Price'] = round((sn_purchases_df['Price']/sn_purchases_df['Count']),2)
sn_purchases_df = sn_purchases_df.drop(columns = ['Purchase ID','Age','Item ID'])
sn_purchases_df = sn_purchases_df.set_index('SN')
sn_purchases_df


,Price,Count,Avg Purchase Price
SN,,,
Lisosia93,18.96,5,3.79
Idastidru52,15.45,4,3.86
Chamjask73,13.83,3,4.61
Iral74,13.62,4,3.40
Iskadarya95,13.10,3,4.37
Ilarin91,12.70,3,4.23
Ialallo29,11.84,3,3.95
Tyidaim51,11.83,3,3.94
Lassilsala30,11.51,3,3.84


## Most Popular Items
- **Identify the 5 most popular items by purchase count, then list (in a table):**
    - Item ID
    - Item Name
    - Purchase Count
    - Item Price
    - Total Purchase Value

In [118]:
#collect most used item
item_occurence_df = df.groupby(['Item ID','Item Name']).count().sort_values('Purchase ID',ascending=False)
item_occurence_df = item_occurence_df.drop(columns=['SN','Age','Gender','Price','bins'])
item_occurence_df = item_occurence_df.rename(columns={'Purchase ID':'Count'})
#collect prices of items
item_prices_df = df[['Item Name','Price', 'Item ID']]
item_prices_df = item_prices_df.groupby(['Item Name','Item ID']).sum()

#merge into new dataframe
most_used_items_df = item_occurence_df.merge(item_prices_df, how='outer', on=['Item Name','Item ID'])
most_used_items_df = most_used_items_df.rename(columns={'Price':'Total Purchase Value'})
#bring back item price
most_used_items_df['Item Price']=round((most_used_items_df['Total Purchase Value']/most_used_items_df['Count']),2)
most_used_items_df.head()



,,Count,Total Purchase Value,Item Price
Item Name,Item ID,,,
"Oathbreaker, Last Hope of the Breaking Storm",178,12,50.76,4.23
Fiery Glass Crusader,145,9,41.22,4.58
"Extraction, Quickblade Of Trembling Hands",108,9,31.77,3.53
Nirvana,82,9,44.10,4.90
"Pursuit, Cudgel of Necromancy",19,8,8.16,1.02


## Most Profitable Items
- **Identify the 5 most profitable items by total purchase value, then list (in a table):**
    - Item ID
    - Item Name
    - Purchase Count
    - Item Price
    - Total Purchase Value

In [119]:
most_used_items_df.sort_values('Total Purchase Value', ascending=False).head()

,,Count,Total Purchase Value,Item Price
Item Name,Item ID,,,
"Oathbreaker, Last Hope of the Breaking Storm",178,12,50.76,4.23
Nirvana,82,9,44.10,4.90
Fiery Glass Crusader,145,9,41.22,4.58
Final Critic,92,8,39.04,4.88
Singed Scalpel,103,8,34.80,4.35


# Observable Trends
- Almost all of the most used items tend to also have the Highest Purchase Value
- More than Half of the users are between the ages of 15-25
- Males spent on average almost 10% less than other kinds of users